#### Portfolio Risk Analysis
long-short corporate bond portfolio

taking only issuer credit risk, hedging with IR, FX, etc

In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy.cluster.hierarchy as sch
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import NonlinearConstraint
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import LinearOperator
import cvxportfolio as cvx

In [136]:
df_port_wgt = pd.read_csv('portfolio_weights.csv', index_col=['ID'])
df_mkt_wgt = pd.read_csv('market_portfolio_weights.csv', index_col=['ID'])
print(len(df_port_wgt), len(df_mkt_wgt))

240 11797


In [137]:
df_factor_exp = pd.read_csv('risk_model_factor_exposure.csv', index_col=['ID'])
df_factor_exp = df_factor_exp.pivot(columns=['FACTOR'],values=['DATA'])
df_factor_exp.columns=df_factor_exp.columns.droplevel(0)
len(df_factor_exp.columns)

Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       455, 456, 457, 458, 459, 460, 461, 462, 498, 502],
      dtype='int64', name='FACTOR', length=206)

In [139]:
df_factor_zoo = pd.read_csv('risk_model_risk_factors.csv', index_col=['FACTOR'])
df_factor_zoo = df_factor_zoo[['HIERARCHY1','HIERARCHY2','NAME']]
set(df_factor_zoo.HIERARCHY1)

{'Risk Type : Equity',
 'Risk Type : FX',
 'Risk Type : Interest Rate',
 'Risk Type : Issuer Credit',
 'Risk Type : Vega'}

In [145]:
df_factor_cov = pd.read_csv('risk_model_factor_covariance.csv')
key1 = "FACTOR_1"
key2 = "FACTOR_2"
value = "DATA"
dict_factor_cov = {
    (df_factor_cov.loc[i,key1],df_factor_cov.loc[i,key2]) : 
    df_factor_cov.loc[i,value] for i in range(len(df_factor_cov))
    }
dict_factor_cov[(2,2)]

0.025747948

In [146]:
df_spec_cov = pd.read_csv('risk_model_specific_variance_covariance.csv')
key1 = "ID_1"
key2 = "ID_2"
value = "DATA"
dict_spec_cov = {
    (df_spec_cov.loc[i,key1],df_spec_cov.loc[i,key2]) : 
    df_spec_cov.loc[i,value] for i in range(len(df_spec_cov))
    }
dict_spec_cov[("AT0000A24UY3","XS2154348424")]

0.000328514

In [147]:
def get_portfolio_risk_exposures(df_wgt, df_risk_exp):
    m_exp = np.matmul(
        np.matrix(df_wgt).T, 
        np.matrix(df_risk_exp.fillna(0).loc[list(df_wgt.index),:])
        )
    return pd.DataFrame(m_exp, columns=df_risk_exp.columns)

In [148]:
df_port_exp = get_portfolio_risk_exposures(df_port_wgt, df_factor_exp)
df_mkt_exp = get_portfolio_risk_exposures(df_mkt_wgt, df_factor_exp)

In [161]:
def get_portfolio_ex_ante_risk(df_exp, dict_factor_cov):
    df_risk_contrib = df_exp * 0.0
    for k in dict_factor_cov.keys():
        tmp_risk = df_exp[k[0]] * dict_factor_cov[k] * df_exp[k[1]]
        df_risk_contrib[k[0]] += tmp_risk
    df_risk_contrib.index = ["Risk"]
    return df_risk_contrib

In [162]:
df_risk_contrib = get_portfolio_ex_ante_risk(df_port_exp, dict_factor_cov)
risk = df_risk_contrib.sum(axis=1)
risk

Risk    0.00989
dtype: float64

In [164]:
def get_portfolio_risk_exp_hedged(df_wgt, df_risk_exp, df_factor_zoo):
    df_exp = get_portfolio_risk_exposures(df_wgt, df_risk_exp)
    credit_risk = list(
        df_factor_zoo[df_factor_zoo.HIERARCHY1=='Risk Type : Issuer Credit'].index
        )
    for k in df_exp.columns:
        if k not in credit_risk:
            df_exp[k] = 0.0
    return df_exp

In [165]:
df_port_exp_hdg = get_portfolio_risk_exp_hedged(df_port_wgt, df_factor_exp, df_factor_zoo)
df_mkt_exp_hdg = get_portfolio_risk_exp_hedged(df_mkt_wgt, df_factor_exp, df_factor_zoo)

In [166]:
df_risk_contrib_hdg = get_portfolio_ex_ante_risk(df_port_exp_hdg, dict_factor_cov)
risk = df_risk_contrib_hdg.sum(axis=1)
risk

Risk    0.00166
dtype: float64

In [172]:
top5_risks = df_risk_contrib_hdg.transpose().sort_values(by='Risk',ascending=False).head(5)
top5_risks.join(df_factor_zoo)

,Risk,HIERARCHY1,HIERARCHY2,NAME
FACTOR,,,,
71,0.000325,Risk Type : Issuer Credit,Credit Quality,Credit Quality - USD Quality Q1
74,0.000247,Risk Type : Issuer Credit,Credit Quality,Credit Quality - USD Quality Q4
73,0.000202,Risk Type : Issuer Credit,Credit Quality,Credit Quality - USD Quality Q3
55,0.000193,Risk Type : Issuer Credit,Credit Style,Credit Style - Momentum
76,0.000161,Risk Type : Issuer Credit,Credit Style,Credit Style - Value


In [169]:
df_risk_contrib_hdg_mkt = get_portfolio_ex_ante_risk(df_mkt_exp_hdg, dict_factor_cov)
risk = df_risk_contrib_hdg_mkt.sum(axis=1)
risk

Risk    0.002352
dtype: float64

In [171]:
top5_risks_mkt = df_risk_contrib_hdg_mkt.transpose().sort_values(by='Risk',ascending=False).head(5)
top5_risks_mkt.join(df_factor_zoo)

,Risk,HIERARCHY1,HIERARCHY2,NAME
FACTOR,,,,
33,0.001261,Risk Type : Issuer Credit,Credit Market Intercept,Credit Market Intercept - Global Market Intercept
5,0.000403,Risk Type : Issuer Credit,Credit Style,Credit Style - Beta
70,0.000391,Risk Type : Issuer Credit,Credit Currency Intercept,Credit Currency Intercept - USD Intercept
69,0.000116,Risk Type : Issuer Credit,Credit Country,Credit Country - United States
66,0.000085,Risk Type : Issuer Credit,Credit Style,Credit Style - Steepness
